# 0. Installation of Faker library

In [39]:
pip install faker

Note: you may need to restart the kernel to use updated packages.


# 1. Imports

In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import random
from faker import Faker
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
Faker.seed(SEED)
fake = Faker()
import warnings
warnings.filterwarnings('ignore')
import plotly.io as pio
pio.renderers.default = "iframe"

# 2. Data Creation

In [41]:
members = []
for i in range(1, 3001):  
    members.append({
        "MemberID": i,
        "Name": fake.name(),
        "Age": random.choices([random.randint(18, 60), None],weights=[0.9, 0.1])[0],
        "Gender": random.choice(["Male", "Female"]),
        "Email": fake.email() if random.random() > 0.1 else None,  
        "JoinDate": fake.date_between(start_date="-2y", end_date="today"),
        "MembershipType": random.choice(["Monthly", "Yearly", "Premium"])  
    })
members_df = pd.DataFrame(members)

In [42]:
trainers = []
for i in range(1, 51):  
    trainers.append({
        "TrainerID": i,
        "Name": fake.name(),
        "Specialty": random.choice(["Yoga", "Cardio", "Strength", "CrossFit", "Pilates"]),
        "Phone": fake.phone_number() if random.random() > 0.05 else None  
    })
trainers_df = pd.DataFrame(trainers)

In [43]:
classes = []
for i in range(1, 201): 
    classes.append({
        "ClassID": i,
        "ClassName": random.choice(["Yoga", "Cardio", "Strength", "CrossFit", "Pilates"]),
        "TrainerID": random.randint(1, 50),
        "Schedule": fake.date_time_between(start_date="-1y", end_date="now"),
        "Notes": fake.sentence() if random.random() > 0.15 else None  
    })
classes_df = pd.DataFrame(classes)

In [44]:
membership_prices = {
    "Monthly": 200,
    "Yearly": 500,
    "Premium": 1000
}

payments = []
for i in range(1, 4001):  
    member_id = random.randint(1, 3000)
    membership_type = members_df.loc[members_df['MemberID'] == member_id, 'MembershipType'].values[0]  
    
    payments.append({
        "PaymentID": i,
        "MemberID": member_id,
        "Amount": membership_prices[membership_type],
        "Method": random.choice(["Cash", "Card",None]),  
        "Date": fake.date_between(start_date="-2y", end_date="today")
    })

payments_df = pd.DataFrame(payments)

In [45]:
attendance = []
for i in range(1, 10001):  
    attendance.append({
        "AttendanceID": i,
        "MemberID": random.randint(1, 3000),
        "ClassID": random.randint(1, 200),
        "Date": fake.date_between(start_date="-1y", end_date="today"),
        "Status": random.choice(["Present", "Absent"])
    })
attendance_df = pd.DataFrame(attendance)

# 3. Data Exploration

In [46]:
dfs = {
    "Members": members_df,
    "Trainers": trainers_df,
    "Classes": classes_df,
    "Attendance": attendance_df,
    "Payments": payments_df
}

for name, df in dfs.items():
    print(f"--- {name} ---")
    print(df.info())
    print(df.describe())
    print('-------------------------------------')


--- Members ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   MemberID        3000 non-null   int64  
 1   Name            3000 non-null   object 
 2   Age             2699 non-null   float64
 3   Gender          3000 non-null   object 
 4   Email           2701 non-null   object 
 5   JoinDate        3000 non-null   object 
 6   MembershipType  3000 non-null   object 
dtypes: float64(1), int64(1), object(5)
memory usage: 164.2+ KB
None
          MemberID          Age
count  3000.000000  2699.000000
mean   1500.500000    38.874027
std     866.169729    12.378949
min       1.000000    18.000000
25%     750.750000    28.000000
50%    1500.500000    38.000000
75%    2250.250000    50.000000
max    3000.000000    60.000000
-------------------------------------
--- Trainers ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries,

# 4. Data Cleaning

In [47]:
for name, df in dfs.items():
    print(df.isnull().sum())
    print('--------------------------')

MemberID            0
Name                0
Age               301
Gender              0
Email             299
JoinDate            0
MembershipType      0
dtype: int64
--------------------------
TrainerID    0
Name         0
Specialty    0
Phone        4
dtype: int64
--------------------------
ClassID       0
ClassName     0
TrainerID     0
Schedule      0
Notes        31
dtype: int64
--------------------------
AttendanceID    0
MemberID        0
ClassID         0
Date            0
Status          0
dtype: int64
--------------------------
PaymentID       0
MemberID        0
Amount          0
Method       1351
Date            0
dtype: int64
--------------------------


In [48]:
members_df['Age']=members_df['Age'].fillna(int(members_df['Age'].mean()))

In [50]:
nan_idx = payments_df["Method"].isna()
payments_df.loc[nan_idx, "Method"] = np.random.choice(
    ["Cash", "Credit"], size=nan_idx.sum(), p=[0.7, 0.3])

In [51]:
members_df.to_csv("members.csv", index=False)
trainers_df.to_csv("trainers.csv", index=False)
classes_df.to_csv("classes.csv", index=False)
payments_df.to_csv("payments.csv", index=False)
attendance_df.to_csv("attendance.csv", index=False)

# 5. Data Visualization

In [52]:
v1=members_df['MembershipType'].value_counts()
plt.pie(v1.values,labels=v1.index,autopct=lambda p: f'{int(p * sum(v1) / 100)}')
plt.title('MembershipType Counts')
plt.show()

In [53]:
sns.histplot(members_df['Age'], bins=10, kde=True)
plt.xlabel('Age')
plt.ylabel('Count')
plt.title('Age Distribution (Histogram + KDE)')
plt.show()

In [54]:
v3=members_df['Gender'].value_counts()
px.pie(v3,names=v3.index,values=v3.values,title='Members by Gender')

In [55]:
v4=trainers_df['Specialty'].value_counts()
px.bar(v4,x=v4.index,y=v4.values,title='Number of trainers in each Speciality',
       text=v4.values,labels={'y': 'Number of Trainers'})

In [56]:
v5=attendance_df['Status'].value_counts()
px.pie(v5,names=v5.index,values=v5.values,title='Status of attendance in classes')

In [57]:
present_att = attendance_df[attendance_df['Status'] == 'Present']
top10_attendance = (
    present_att.groupby("MemberID")['AttendanceID']
    .count()
    .reset_index(name="TotalPresent")
    .sort_values("TotalPresent", ascending=False)
    .head(10)
)

top10_attendance = top10_attendance.merge(
    members_df[['MemberID', 'Name']], 
    on="MemberID", 
    how="left"
)

px.bar(
    top10_attendance,
    x="Name",
    y="TotalPresent",
    text="TotalPresent",
    title="Top 10 Members by Attendance"
)


In [58]:
payments_with_members = payments_df.merge(members_df, on="MemberID", how="inner")

membership_revenue = (
    payments_with_members.groupby("MembershipType")["Amount"]
    .sum()
    .reset_index()
    .sort_values("Amount", ascending=False)
)
px.bar(
    membership_revenue,
    x="MembershipType",
    y="Amount",
    title="Revenue by Membership Type",
    labels={"MembershipType": "Subscription Type", "Amount": "Total Revenue"},
    text="Amount"
)
